# VAE on ModelNet for scene completion

Imports and setup

In [1]:
import numpy as np
import torch
import sys
import os
from torch.utils.data import DataLoader


In [2]:
sys.path.append('/home/benjamin.deporte/MVA/MVA_ModelNet_VAE/')

from libs import dataset

Import dataset and form dataloader

In [32]:
# uses datasets and Dataset class from https://github.com/antao97/PointCloudDatasets/

# parameters :
# root : root directory, must contain file dataset_name + '_hdf5_2048
# dataset_name : modelnet10 par défaut, sinon modelnet40
# class_choice : permet de ne sélectionner qu'une classe
# split : 'train' par défaut. valeurs possibles = 'test', 'trainval', 'all', etc.
# possibilité de data augmentation également

modelnet10 = dataset.Dataset(root='/home/benjamin.deporte/MVA/MVA_ModelNet_VAE/data/', load_name=True)
modelnet40 = dataset.Dataset(root='/home/benjamin.deporte/MVA/MVA_ModelNet_VAE/data/', dataset_name='modelnet40')

In [33]:
print(f"ModelNet10 dataset has {len(modelnet10)} elements")
print(f"ModelNet40 dataset has {len(modelnet40)} elements")

ModelNet10 dataset has 3991 elements
ModelNet40 dataset has 9843 elements


In [43]:
batch_size = 1

In [44]:
mn10_loader = DataLoader(
    modelnet10,
    batch_size=batch_size,
    shuffle=True
    )

mn10_iter = iter(mn10_loader)

In [45]:
# outputs :

point_set, label, name, file = next(mn10_iter)

print(f"point set = {point_set.shape}")
print(f"label = {label}")
print(f"name = {name}")
print(f"file = {file}")

point set = torch.Size([1, 2048, 3])
label = tensor([[3]])
name = ('desk',)
file = ('desk/desk_0164.ply',)


Some visualisation

In [46]:
from libs import visualize

In [50]:
item = 0
point_set, label, name, file = next(mn10_iter)

print(point_set.size(), point_set.type(), label.size(), label.type(), name)
dataset_name = 'modelnet10'
save_root = os.path.join("/home/benjamin.deporte/MVA/MVA_ModelNet_VAE/image/", dataset_name)
split='train'
name = 'essai'

path = os.path.join(save_root, dataset_name + '_' + split + str(item) + '_' + str(name) + '.xml')
visualize.mitsuba(point_set.numpy(), path)

torch.Size([1, 2048, 3]) torch.FloatTensor torch.Size([1, 1]) torch.LongTensor ('sofa',)
